In [239]:
import time
import numpy as np
import sys
import requests
import json
from random import sample
from math import ceil

In [2]:
def delay_print(s):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(0.05)

In [3]:
def api_call(url):
    """Makes api call to the Pokemon type endpoint and returns json text"""
    response = requests.get(url)
    response_json = json.loads(response.text)
    return response_json


In [250]:
class Pokemon:
    def __init__(self, name, speed, health='=========='):
        self.name = name
        self.types = []
        self.moves = []
        self.health = health
        self.bars = 30
        self.speed = speed
        self.pokemon_url = 'https://pokeapi.co/api/v2/pokemon/'+self.name+'/'
        self.weaknesses = []
        self.resistances = []
        self.move_dict = {}


    def get_moves(self): 
        """Creates a list of moves for battle. The moves are random
        assignments from all available moves in the api"""
        response_json = api_call(self.pokemon_url)
        num_of_moves = len(response_json['moves'])
        # randomly selects 8 out of all moves available
        # 8 because some moves have 0 attack which is lame
        # and I want to filter down to 4 useable moves 
        move_index = sample(range(num_of_moves), 8) 

        for i in move_index:
            self.moves.append(
                response_json['moves'][i]['move']['name'])


    def poke_types(self):
        """Updates types list for the Pokemon's type(s)"""
        self.types.clear() #empty the list so that duplicates are not created if ran multiple times
        response_json = api_call(self.pokemon_url)
        for i in range(len(response_json['types'])):
            type_resp = response_json['types'][i]['type']['name']
            self.types.append(type_resp)


    def factors(self):
        """Finds the weaknesses and resistance types for this Pokemon"""
        self.poke_types()
        for i in self.types:
            type_url = 'https://pokeapi.co/api/v2/type/'+i+'/'
            type_response_json = api_call(type_url)

            for j in range(len(type_response_json['damage_relations']['double_damage_from'])):
                self.weaknesses.append(type_response_json['damage_relations']['double_damage_from'][j]['name'])

            for k in range(len(type_response_json['damage_relations']['half_damage_from'])):
                self.resistances.append(type_response_json['damage_relations']['half_damage_from'][k]['name'])

            self.weaknesses = np.unique(np.array(self.weaknesses)).tolist()
            self.resistances = np.unique(np.array(self.resistances)).tolist()


    def delete_no_damage(self):
        """
        Removes any move with a 0 damage value from the move_dict and moves
        """
        del_list = []
        for m in self.move_dict:
            if self.move_dict[m]['power'] == 0:
                del_list.append(m)
        for n in del_list:
            del self.move_dict[n]

        #remove excess moves from move_dict
        for k,v in enumerate(self.move_dict.copy()):
            if k > 3:
                del self.move_dict[v]

        #remove the extra moves from moves
        self.moves = self.move_dict.keys()
    
    def move_info(self): 
        """Finds type and power attributes for each of the pokemons moves.  The results
        fill in the move_dict attribute. """
        for moves in self.moves:
            move = moves.replace(' ', '-').lower()
            url = 'https://pokeapi.co/api/v2/move/'+move+'/'
            response_json = api_call(url)

            move_type = response_json['type']['name']
            move_power = 0 if response_json['power'] == None else response_json['power'] // 10
            self.move_dict[move] = {'type' : move_type, 'power' : move_power}

        self.delete_no_damage()


    def get_attrs(self):
        """Retrieves all the pokemons attributes to prepare for battle"""
        self.get_moves()
        self.poke_types()
        self.factors()
        self.move_info()

    
def damage_multiplier(defending_mon, quicker_mon, move): 
    d_resistences = defending_mon.resistances
    d_weaknesses = defending_mon.weaknesses
    m_type = quicker_mon.move_dict[move].get('type')
    # move_types = set(m_type)
    
    if m_type in d_weaknesses:
        multiplier = 2
    elif m_type in d_resistences:
        multiplier = .5
    else: 
        multiplier = 1
    
    return multiplier


def damage_calc(attacking_mon, defending_mon, index):
    """Calculates the actual damage done by selected move during battle"""
    dmg = int(attacking_mon.move_dict[list(attacking_mon.moves)[index-1]]['power'] * damage_multiplier(defending_mon, 
                                                                                        attacking_mon, 
                                                                                        list(attacking_mon.moves)[index-1]))
    return dmg

In [240]:
def battle_turn1(quicker_mon, slower_mon):
    print("Go", quicker_mon.name, "!")
    
    for i, x in enumerate(quicker_mon.moves):
        print(i+1, x)
    index = int(input("Pick a move: "))
    
    print(quicker_mon.name ,"used", list(quicker_mon.moves)[index-1])
    time.sleep(1)
    print(quicker_mon.name, 
        "did", 
        damage_calc(quicker_mon, slower_mon, index),
        "damage!")

    slower_mon.bars -= damage_calc(quicker_mon, slower_mon, index)
    slower_mon.health = ""

    # for i in range(int(slower_mon.bars)):
    #     slower_mon.health += "="
    slower_mon.health = '=' * ceil(slower_mon.bars / 3)

    print(quicker_mon.name ,"health:", quicker_mon.health)
    print(slower_mon.name ,"health:", slower_mon.health)

In [241]:
def battle_turn2(quicker_mon, slower_mon):
    print("Go", slower_mon.name, "!")
    for i, x in enumerate(slower_mon.moves):
        print(i+1, x)

    index = int(input("Pick a move: "))
    
    print(slower_mon.name ,"used", list(slower_mon.moves)[index-1])
    time.sleep(1)
    print(slower_mon.name,
        "did",
        damage_calc(slower_mon, quicker_mon, index),
        "damage!")

    quicker_mon.bars -= damage_calc(slower_mon, quicker_mon, index)
    quicker_mon.health = ""


    # for i in range(int(quicker_mon.bars)):
    #     quicker_mon.health += "="
    quicker_mon.health = '=' * ceil(quicker_mon.bars / 3 )
        
    print(slower_mon.name ,"health:", slower_mon.health)
    print(quicker_mon.name ,"health:", quicker_mon.health)            

In [7]:
def battle(Pokemon1, Pokemon2): 
    
    print("-----POKEMON BATTLE-----")
    print("Pokemon 1:", Pokemon1.name)
    print("TYPE/", Pokemon1.types)
    print("\nVS\n")
    print("Pokemon 2:", Pokemon2.name)
    print("TYPE/", Pokemon2.types, '\n')
    
#evaluate speed of pokemon
    
    while Pokemon1.speed > Pokemon2.speed == True:
        quicker_mon = Pokemon1
        slower_mon = Pokemon2
    
    else: 
        slower_mon = Pokemon1
        quicker_mon = Pokemon2
        
    while (quicker_mon.bars > 0) and (slower_mon.bars > 0):
            
            battle_turn1(quicker_mon, slower_mon)
            
            if slower_mon.bars <= 0:
                delay_print("\n..." + slower_mon.name + ' fainted.')
                break
                
 #mirroring the attack pattern of the quicker mon but for the slower mon

            battle_turn2(quicker_mon, slower_mon)           
            
            if quicker_mon.bars <= 0:
                delay_print("\n..." + quicker_mon.name + ' fainted.')
                break

In [251]:
if __name__ == '__main__':
    # gyarados = Pokemon('gyarados', 'water', {'crunch': 1, 'surf': 3}, 4)
    # aerodactyl = Pokemon('aerodactyl', 'rock', {'air-cutter': 2, 'head-smash': 3}, 7)

    gyarados = Pokemon('gyarados', 4)
    gyarados.get_attrs()
    aerodactyl = Pokemon('aerodactyl', 7)
    aerodactyl.get_attrs()
#could use api calls for move type and move power instead of using dictionaries
    #for damage calcs, could use power % 10 to simplify health ga

In [252]:
gyarados.move_dict


{'stone-edge': {'type': 'rock', 'power': 10},
 'thrash': {'type': 'normal', 'power': 12},
 'brine': {'type': 'water', 'power': 6},
 'thunderbolt': {'type': 'electric', 'power': 9}}

In [253]:
aerodactyl.move_dict

{'dragon-pulse': {'type': 'dragon', 'power': 8},
 'fly': {'type': 'flying', 'power': 9},
 'assurance': {'type': 'dark', 'power': 6},
 'thief': {'type': 'dark', 'power': 6}}

In [255]:
battle(gyarados, aerodactyl)

-----POKEMON BATTLE-----
Pokemon 1: gyarados
TYPE/ ['water', 'flying']

VS

Pokemon 2: aerodactyl
TYPE/ ['rock', 'flying'] 

Go aerodactyl !
1 dragon-pulse
2 fly
3 assurance
4 thief
aerodactyl used dragon-pulse
aerodactyl did 8 damage!
aerodactyl health: ====
gyarados health: =====
Go gyarados !
1 stone-edge
2 thrash
3 brine
4 thunderbolt
gyarados used brine
gyarados did 12 damage!
gyarados health: =====
aerodactyl health: 

...aerodactyl fainted.

In [29]:
ivysaur.get_moves() #updates moves to new choices but now has no damage associated 
ivysaur.moves 

['bullet-seed', 'fury-cutter', 'grassy-terrain', 'nature-power']

In [23]:
gyarados.moves_list #just a list of the random moves. work in progress

['aqua-tail', 'outrage', 'scale-shot', 'thunder-wave']

In [30]:

for move in ivysaur.moves:
    move_type, move_power = move_info(move)
    ivysaur.move_dict[move] = {'type' : move_type, 'power' : move_power}
    

In [31]:
ivysaur.move_dict

{'bullet-seed': {'type': 'grass', 'power': 25},
 'fury-cutter': {'type': 'bug', 'power': 40},
 'grassy-terrain': {'type': 'grass', 'power': 0},
 'nature-power': {'type': 'normal', 'power': 0}}

In [ ]:
gyarados.types

In [ ]:
gyarados.poke_types()
gyarados.types_list

In [27]:
ivysaur = Pokemon('ivysaur', 'grass', 'na', 4,)

In [ ]:
ivysaur.get_moves()

In [ ]:
ivysaur.poke_types()

In [ ]:
ivysaur.types_list

In [ ]:
ivysaur.moves